# Geoqueries en Mongo

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#¿Qué-podemos-hacer-con-las-geoqueries?" data-toc-modified-id="¿Qué-podemos-hacer-con-las-geoqueries?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>¿Qué podemos hacer con las geoqueries?</a></span></li><li><span><a href="#Generamos-la-conexión-con-Mongo" data-toc-modified-id="Generamos-la-conexión-con-Mongo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generamos la conexión con Mongo</a></span></li><li><span><a href="#Importar-colecciones" data-toc-modified-id="Importar-colecciones-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importar colecciones</a></span></li><li><span><a href="#Crear-indexes" data-toc-modified-id="Crear-indexes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Crear indexes</a></span></li><li><span><a href="#Empezamos-con-las-Geoqueries" data-toc-modified-id="Empezamos-con-las-Geoqueries-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Empezamos con las Geoqueries</a></span></li><li><span><a href="#Intersects" data-toc-modified-id="Intersects-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Intersects</a></span></li><li><span><a href="#GeoWithin" data-toc-modified-id="GeoWithin-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>GeoWithin</a></span></li><li><span><a href="#Near" data-toc-modified-id="Near-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Near</a></span></li><li><span><a href="#Extra,-extra!!-mis-datos-extraídos-de-Foursquare-en-Mongo" data-toc-modified-id="Extra,-extra!!-mis-datos-extraídos-de-Foursquare-en-Mongo-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Extra, extra!! mis datos extraídos de Foursquare en Mongo</a></span><ul class="toc-item"><li><span><a href="#Voy-a-traerme-todas-las-fruterías-de-Madrid" data-toc-modified-id="Voy-a-traerme-todas-las-fruterías-de-Madrid-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Voy a traerme todas las fruterías de Madrid</a></span></li></ul></li><li><span><a href="#Vamos-a-buscar-a-cuántos-km-están-las-fruterías-de-mi-casa" data-toc-modified-id="Vamos-a-buscar-a-cuántos-km-están-las-fruterías-de-mi-casa-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Vamos a buscar a cuántos km están las fruterías de mi casa</a></span></li></ul></div>

## ¿Qué podemos hacer con las geoqueries?
Con las geoquerías podemos responder a preguntas como las siguientes
 * ¿Dónde están las farmacias más cercanas a mi ubicación?
 * ¿Qué restaurantes hay en esta poligonal?

Cada punto de la Tierra se caracteriza por dos números:
 * Longitud: ángulo respecto al meridiano de Greenwich. Va de -180° (antiGreenwich) a +180° (también antiGreenwich)
 * Latitud: ángulo con respecto al ecuador. Va de -90° (sur) a +90° (norte)

![latierra](../images/latlon.gif)

## Generamos la conexión con Mongo

In [ ]:
from pymongo import MongoClient
conn = MongoClient("localhost:27017")
db = conn.get_database("ironhack")

## Importar colecciones     
Sigamos la documentación oficial de Mongo: [Mongo Geo Example](https://docs.mongodb.com/manual/tutorial/geospatial-tutorial/)
Creamos estas dos colecciones nuevas:
 * `restaurants2`
 * `neighborhoods`

## Crear indexes
La indexación geoespacial de MongoDB permite   
ejecutar eficientemente **consultas espaciales** sobre una colección que contiene formas y puntos geoespaciales.
Vamos a crear un índice geográfico para que Mongo sepa que queremos hacer geo-consultas en esta colección.  
Esto sólo debe hacerse una vez, y ahora la colección de Mongo está correctamente indexada para siempre.

In [ ]:
from pymongo import GEOSPHERE

## Empezamos con las Geoqueries   
¿En qué barrio estoy?      
¿Qué elementos **intersecan** el elemento dado?

## Intersects

In [ ]:
n = db.get_collection("neighborhoods")
r = db.get_collection("restaurants2")

In [ ]:
coordenadas = [-73.93, 40.82]

In [ ]:
proj = {"_id":0, "name": 1}

## GeoWithin

¿Qué elementos están **contenidos** en el elemento dado?     
¿Qué restaurantes hay en este barrio (polígono)?

## Near

¿Qué elementos se encuentran a una distancia de una geometría determinada?     
`$nearSphere` y `$maxDistance` nos ayudan a encontrar todos los elementos dentro de `maxDistance` metros de la geometría deseada, **ordenados** de más cercano a más lejano.

In [ ]:
coordenadas = [-73.93, 40.82]

In [ ]:
# Km que quiero  caminar
kms = 0.3

## Extra, extra!! mis datos extraídos de Foursquare en Mongo

### Voy a traerme todas las fruterías de Madrid

In [ ]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

In [ ]:
tok1= os.getenv("tok1")
tok2= os.getenv("tok2")

In [ ]:
madrid = {'type': 'Point', 'coordinates': [-3.6793, 40.42955]}

In [ ]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [ ]:
parametros = {
    "client_id": tok1,
    "client_secret": tok2,
    "v": "20180323",
    "ll": f"{madrid.get('coordinates')[1]},{madrid.get('coordinates')[0]}",
    "query": "frutería", 
    "limit": 100    
}

In [ ]:
resp = requests.get(url= url_query, params = parametros).json()

In [ ]:
data = resp.get("response").get("groups")[0].get("items")

In [ ]:
from functools import reduce
import operator

In [ ]:
# Mis fuentes --stackoverflow
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [ ]:
mapa_nombre =  ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]

In [ ]:
lista_frutas = []
for dic in data:
    paralista = {}
    paralista["name"] = getFromDict(dic, mapa_nombre)
    paralista["latitud"]= getFromDict(dic, mapa_latitud)
    paralista["longitud"] = getFromDict(dic,mapa_longitud)
    lista_frutas.append(paralista)

In [ ]:
df = pd.DataFrame(lista_frutas)
df.head()

Vamos a manipular los datos para tener esta misma estructura en tipo point y poder hacer un index en Mongo para realizar geoqueries

In [ ]:
lista_frutas[0]

## Vamos a buscar a cuántos km están las fruterías de mi casa